For this workshop, you need:

* An Azure Machine Learning workspace. 
* The Azure Machine Learning Python SDK v2 installed. 

To install the SDK you can either,

Create a compute instance, which already has installed the latest AzureML Python SDK and is pre-configured for ML workflows.

Use the followings commands to install Azure ML Python SDK v2:

```bash
conda activate <virtual_env_name>
pip install azure-ai-ml==1.0.0
```

If you're using a virtual env, make sure to install the sdk inside the virtual env.

The virtual environment for sdkv2 on Azure Notebooks is called `azureml_py310_sdkv2`.


## Connect to ML Client

To connect to a workspace, you need to provide a subscription, resource group and workspace name. These details are used in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace.

In the following example, the default Azure authentication is used along with the default workspace configuration or from any `config.json` file you might have copied into the folders structure. If no `config.json` is found, then you need to manually introduce the subscription_id, resource_group and workspace when creating `MLClient`.

```python
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AzureML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AZUREML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
```


In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Add config.json file to the workspace
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential, path="config.json")

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/jcharley4/code/Users/jcharley/config.json


# Model Training

## 1. Create Managed Compute

A compute is a designated compute resource where you run your job or host your endpoint. Azure Machine learning supports the following types of compute:

- **Compute instance** - a fully configured and managed development environment in the cloud. You can use the instance as a training or inference compute for development and testing. It's similar to a virtual machine on the cloud.

- **Compute cluster** - a managed-compute infrastructure that allows you to easily create a cluster of CPU or GPU compute nodes in the cloud.

- **Inference cluster** - used to deploy trained machine learning models to Azure Kubernetes Service. You can create an Azure Kubernetes Service (AKS) cluster from your Azure ML workspace, or attach an existing AKS cluster.

- **Attached compute** - You can attach your own compute resources to your workspace and use them for training and inference.

You can create a compute using the Studio, the cli and the sdk.

<hr>

In [2]:
from azure.ai.ml.entities import AmlCompute

my_cluster = AmlCompute(
    name="cpu-cluster-CA",
    type="amlcompute", 
    size="STANDARD_DS3_V2", 
    min_instances=0, 
    max_instances=2,
    location="westeurope", 	
)

ml_client.compute.begin_create_or_update(my_cluster)


## 2. Register File Data Asset

**Datastore** - Azure Machine Learning Datastores securely keep the connection information to your data storage on Azure, so you don't have to code it in your scripts.

An Azure Machine Learning datastore is a **reference** to an **existing** storage account on Azure. The benefits of creating and using a datastore are:
* A common and easy-to-use API to interact with different storage type. 
* Easier to discover useful datastores when working as a team.
* When using credential-based access (service principal/SAS/key), the connection information is secured so you don't have to code it in your scripts.

Supported Data Resources: 

* Azure Storage blob container
* Azure Storage file share
* Azure Data Lake Gen 1
* Azure Data Lake Gen 2
* Azure SQL Database 
* Azure PostgreSQL Database
* Azure MySQL Database

It is not a requirement to use Azure Machine Learning datastores - you can use storage URIs directly assuming you have access to the underlying data.

You can create a datastore using the Studio, the cli and the sdk.

<hr>



**Data asset** - Create data assets in your workspace to share with team members, version, and track data lineage.

By creating a data asset, you create a reference to the data source location, along with a copy of its metadata. 

The benefits of creating data assets are:

* You can **share and reuse data** with other members of the team such that they do not need to remember file locations.
* You can **seamlessly access data** during model training (on any supported compute type) without worrying about connection strings or data paths.
* You can **version** the data.

<hr>


In [3]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path="../data/Day1-exercice5-Taxi/taxi-data.csv",
    type=AssetTypes.URI_FILE, # URI_FOLDER
    description="Taxi dataset",
    name="taxi-data"
)
ml_client.data.create_or_update(my_data)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'taxi-data', 'description': 'Taxi dataset', 'tags': {}, 'properties': {}, 'id': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Microsoft.MachineLearningServices/workspaces/test_learn/data/taxi-data/versions/5', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/jcharley4/code/Users/jcharley/VBD_Day1/correction', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f27780a4e80>, 'serialize': <msrest.serialization.Serializer object at 0x7f27780a4f70>, 'version': '5', 'latest_version': None, 'path': 'azureml://subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourcegroups/Learning/workspaces/test_learn/datastores/workspaceblobstore/paths/LocalUpload/2e56e9007690a9db90f90b8830ddcde4/taxi-data.csv', 'datastore': Non

## 3. Register Train Environment

Azure Machine Learning environments define the execution environments for your **jobs** or **deployments** and encapsulate the dependencies for your code. 

Azure ML uses the environment specification to create the Docker container that your **training** or **scoring code** runs in on the specified compute target.

Create an environment from a
* conda specification
* Docker image
* Docker build context

There are two types of environments in Azure ML: **curated** and **custom environments**. Curated environments are predefined environments containing popular ML frameworks and tooling. Custom environments are user-defined.

<hr>

We can register an **environment** with cli v2 or sdk v2 using the following syntax:


In [7]:
from azure.ai.ml.entities import Environment

my_environment = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src-exercice6/environment/train-conda.yml",
    name="taxi-train-env",
    description="Environment created from a Docker image plus Conda environment to train taxi model.",
)

ml_client.environments.create_or_update(my_environment)

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'taxi-train-env', 'description': 'Environment created from a Docker image plus Conda environment to train taxi model.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Microsoft.MachineLearningServices/workspaces/test_learn/environments/taxi-train-env/versions/13', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/jcharley4/code/Users/jcharley/VBD_Day1/correction', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f2753184d90>, 'serialize': <msrest.serialization.Serializer object at 0x7f27705fdb20>, 'version': '13', 'latest_version': None, 'conda_file': {'channels': ['defaults', 'anaconda', 'conda-forge'], 'dependencies': ['python=3.7.5', 'pip', {'pip': ['azureml-mlflow==1.38.0', 'azure-ai-ml==1.0.0', 'pyarrow==10.0.0', 'ruamel.yaml==0.17.21', 'scikit-learn==0.24.1'

## BUILD PIPELINE

In [16]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input, Output, load_component
from azure.ai.ml.constants import AssetTypes, InputOutputModes

# Create pipeline job
parent_dir = "./src-exercice7/components"

# 1. Load components
prepare_data = load_component(source=parent_dir + "/prep.yml")
train_model = load_component(source=parent_dir + "/train.yml")
evaluate_model = load_component(source=parent_dir + "/evaluate.yml")
register_model = load_component(source=parent_dir + "/register.yml")

# 2. Construct pipeline
@pipeline()
def taxi_training_pipeline(raw_data, enable_monitoring, table_name):
    
    prepare = prepare_data(
        raw_data=raw_data,
        enable_monitoring=enable_monitoring, 
        table_name=table_name
    )

    train = train_model(
        train_data=prepare.outputs.train_data
    )

    evaluate = evaluate_model(
        model_name="taxi-model",
        model_input=train.outputs.model_output,
        test_data=prepare.outputs.test_data
    )


    register = register_model(
        model_name="taxi-model",
        model_path=train.outputs.model_output,
        evaluation_output=evaluate.outputs.evaluation_output
    )

    return {
        "pipeline_job_train_data": prepare.outputs.train_data,
        "pipeline_job_test_data": prepare.outputs.test_data,
        "pipeline_job_trained_model": train.outputs.model_output,
        "pipeline_job_score_report": evaluate.outputs.evaluation_output,
    }


pipeline_job = taxi_training_pipeline(
    Input(type=AssetTypes.URI_FILE, path="taxi-data@latest"), "false", "taximonitoring"
)

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster-CA"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

In [17]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="Training-sdkV2"
)
pipeline_job

Uploading prep (0.01 MBs): 100%|██████████| 10020/10020 [00:00<00:00, 258277.57it/s]


